# Majority model validation with `bsky-net`

(This is a work in progress)

Disclaimer: this example currently uses _fake data labels_. Specifically, the "on-topic" and "expressed opinion" labels are randomly generated. However, this example is representative of how this validation set could be used.


In [19]:
# Imports
import json
import random
import os

from bsky_net import did_from_uri, UserTimestep, generate_timestamps

# Constants
ENGAGEMENT_DIR = "../data/processed/engagement-daily-2023-04-01"
START_DATE = "2022-11-17"  # Bluesky launch date
END_DATE = "2023-02-01"

In [21]:
# Check out data

for timestamp in generate_timestamps(START_DATE, END_DATE):
    with open(os.path.join(ENGAGEMENT_DIR, f"{timestamp}.json"), "r") as f:
        data = json.load(f)
        print(json.dumps(data, indent=2))
        break


{
  "did:plc:ragtjsm2j2vknwkz3zp4oxrd": {
    "interactive": true,
    "consumed": {
      "at://did:plc:vpkhqolt662uhesyj6nxm7ys/app.bsky.feed.post/3jho2rgvx7c27": {
        "did": "did:plc:vpkhqolt662uhesyj6nxm7ys",
        "uri": "at://did:plc:vpkhqolt662uhesyj6nxm7ys/app.bsky.feed.post/3jho2rgvx7c27",
        "in_network": false,
        "createdAt": "2022-11-17T01:05:19.257Z",
        "expressed_opinion": 0,
        "reactions": [
          {
            "type": "like",
            "createdAt": "2022-11-17T01:05:55.172Z"
          }
        ]
      },
      "at://did:plc:yk4dd2qkboz2yv6tpubpc6co/app.bsky.feed.post/3jho3ajhbbk2c": {
        "did": "did:plc:yk4dd2qkboz2yv6tpubpc6co",
        "uri": "at://did:plc:yk4dd2qkboz2yv6tpubpc6co/app.bsky.feed.post/3jho3ajhbbk2c",
        "in_network": true,
        "createdAt": "2022-11-17T01:13:45.284Z",
        "expressed_opinion": 1,
        "reactions": [
          {
            "type": "like",
            "createdAt": "2022-11-17T01:16:

In [22]:
# Initialize opinions
opinions = {}

# Run majority model
for timestamp in generate_timestamps(START_DATE, END_DATE):
    with open(os.path.join(ENGAGEMENT_DIR, f"{timestamp}.json"), "r") as f:
        data: dict[str, UserTimestep] = json.load(f)

        # TODO: define majority rules

In [24]:
# TODO: Visualize opinions over time, etc.

In [17]:
# TODO: Actual validation analysis, calculate metrics, etc.

positive_count = sum(1 for opinion in opinions.values() if opinion > 0)
negative_count = sum(1 for opinion in opinions.values() if opinion < 0)
neutral_count = sum(1 for opinion in opinions.values() if opinion == 0)

print(f"Number of users: {len(opinions)}")
print(f"Positive opinions: {positive_count}")
print(f"Negative opinions: {negative_count}")
print(f"Neutral opinions: {neutral_count}")

Number of users: 191
Positive opinions: 191
Negative opinions: 0
Neutral opinions: 0
